In [13]:
import pandas as pd

# read in labels to split test/train set

palsa_labels = pd.read_csv("/home/nadjaflechner/Palsa_data/dataset_100m/palsa_labels.csv", index_col= 0)
palsa_labels.loc[palsa_labels['palsa_percentage'] > 0, 'palsa'] = 1
palsa_labels.loc[palsa_labels['palsa_percentage'] == 0, 'palsa'] = 0

binary_df = palsa_labels.drop('palsa_percentage', axis = 1)
binary_df.to_csv("/home/nadjaflechner/Palsa_data/binary_palsa_labels_100m.csv")


# train = palsa_labels.sample(frac=0.8,random_state=200)
# test = palsa_labels.drop(train.index)

# # train = palsa_labels[:11000]
# # test = palsa_labels[11000:14000]

# train.to_csv("/home/nadjaflechner/Palsa_data/train_100mtest.csv")
# test.to_csv("/home/nadjaflechner/Palsa_data/test100mtesst.csv")

In [3]:
palsa_labels.shape

(67876, 2)

In [4]:
67876*0.8

54300.8

In [16]:
palsa_labels.head(4)

,palsa_percentage,palsa
758_66_55_2018_crop_8311,8,1.0
758_66_55_2018_crop_8315,36,1.0
758_66_55_2018_crop_8316,9,1.0
758_66_55_2018_crop_8318,0,0.0


In [17]:
import os 

binary_df = palsa_labels.drop('palsa_percentage', axis = 1)
binary_df.to_csv("/Users/nadja/Documents/UU/Thesis/generated_tifs/binary_palsa_labels.csv")


In [12]:
########################################
## WHEN RUNNING MUST BE IN TORCH ENV ##
########################################

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import rasterio
import numpy as np

# Define the custom dataset
class ImageDataset(Dataset):
    def __init__(self, image_dir, labels_df):
        self.image_dir = image_dir
        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.index[idx]
        img_path = os.path.join(self.image_dir, f"{img_name}.tif")

        # Open the TIF image using rasterio
        with rasterio.open(img_path) as src:
            # Read the image data
            image_data = src.read()
        image_array = np.array(image_data)
        image_tensor = torch.from_numpy(image_array)
        image_tensor = image_tensor.float()

        label = self.labels_df.iloc[idx, 0]

        return image_tensor, label

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 50 * 50, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Set the paths and parameters
image_dir = "/home/nadjaflechner/Palsa_data/dataset_100m/"
labels_file = "/home/nadjaflechner/Palsa_data/train_100mtest.csv"
batch_size = 40
num_epochs = 5
learning_rate = 0.001

# Load the labels from the CSV file
labels_df = pd.read_csv(labels_file, index_col=0)

# Create the dataset and data loader
dataset = ImageDataset(image_dir, labels_df)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Create the CNN model
model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "cnn_model.pth")


IndexError: Target 23 is out of bounds.

In [21]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import rasterio
import numpy as np

# Define the custom dataset
class ImageDataset(Dataset):
    def __init__(self, image_dir, labels_df):
        self.image_dir = image_dir
        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.index[idx]
        img_path = os.path.join(self.image_dir, f"{img_name}.tif")

        # Open the TIF image using rasterio
        with rasterio.open(img_path) as src:
            # Read the image data
            image_data = src.read()
        image_array = np.array(image_data)
        image_tensor = torch.from_numpy(image_array)
        image_tensor = image_tensor.float()

        label = self.labels_df.iloc[idx, 0]

        return image_tensor, label

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 50 * 50, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        # x = self.conv3(x)
        # x = self.relu(x)
        # x = self.maxpool(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = self.fc(x)
        return x


# Set the paths and parameters
image_dir = "/home/nadjaflechner/Palsa_data/dataset_100m/"
labels_file = "/home/nadjaflechner/Palsa_data/binary_palsa_labels_100m.csv"
batch_size = 40
num_epochs = 10
learning_rate = 0.001

# Load the labels from the CSV file
labels_df = pd.read_csv(labels_file, index_col=0)

# Split the dataset into training and validation sets
train_df = labels_df.sample(frac=0.8,random_state=200)
val_df = labels_df.drop(train_df.index)

# Create the datasets and data loaders
train_dataset = ImageDataset(image_dir, train_df)
val_dataset = ImageDataset(image_dir, val_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Create the CNN model
model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), "cnn_model.pth")

Epoch [1/10], Loss: 20.4804
Epoch [2/10], Loss: 5.1656
Epoch [3/10], Loss: 1.1930
Epoch [4/10], Loss: 0.6793
Epoch [5/10], Loss: 0.2312
Epoch [6/10], Loss: 0.4751
Epoch [7/10], Loss: 1.3029
Epoch [8/10], Loss: 0.4788
Epoch [9/10], Loss: 0.3540
Epoch [10/10], Loss: 0.1911
Validation Accuracy: 82.35%


In [19]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import rasterio
import numpy as np

# Define the custom dataset
class ImageDataset(Dataset):
    def __init__(self, image_dir, labels_df):
        self.image_dir = image_dir
        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.index[idx]
        img_path = os.path.join(self.image_dir, f"{img_name}.tif")

        # Open the TIF image using rasterio
        with rasterio.open(img_path) as src:
            # Read the image data
            image_data = src.read()
        image_array = np.array(image_data)
        image_tensor = torch.from_numpy(image_array)
        image_tensor = image_tensor.float()

        label = self.labels_df.iloc[idx, 0]

        return image_tensor, label

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 25 * 25, 500)
        self.fc2 = nn.Linear(500, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        # print(x.shape)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sig(x)
        return x


# Set the paths and parameters
image_dir = "/Users/nadja/Documents/UU/Thesis/generated_tifs/"
labels_file = "/Users/nadja/Documents/UU/Thesis/generated_tifs/binary_palsa_labels.csv"
batch_size = 40
num_epochs = 10
learning_rate = 0.001

# Load the labels from the CSV file
labels_df = pd.read_csv(labels_file, index_col=0)

# Split the dataset into training and validation sets
train_df = labels_df.sample(frac=0.8,random_state=200)
val_df = labels_df.drop(train_df.index)

# Create the datasets and data loaders
train_dataset = ImageDataset(image_dir, train_df)
val_dataset = ImageDataset(image_dir, val_df)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Create the CNN model
model = CNN()

# Define the loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(dim=1))
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the model on the validation set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), "cnn_model.pth")

Epoch [1/10], Loss: 60.0000
Epoch [2/10], Loss: 40.0000
Epoch [3/10], Loss: 40.0000
Epoch [4/10], Loss: 40.0000
Epoch [5/10], Loss: 40.0000
Epoch [6/10], Loss: 80.0000
Epoch [7/10], Loss: 40.0000
Epoch [8/10], Loss: 20.0000
Epoch [9/10], Loss: 20.0000
Epoch [10/10], Loss: 60.0000
Validation Accuracy: 54.90%


In [20]:
predicted

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
for images, labels in train_dataloader:
    print(labels, type(labels))


tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
        0., 0., 0., 0.], dtype=torch.float64) <class 'torch.Tensor'>
tensor([1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1.], dtype=torch.float64) <class 'torch.Tensor'>
tensor([0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1.], dtype=torch.float64) <class 'torch.Tensor'>
tensor([1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1.,
        0., 0., 0., 1.], dtype=torch.float64) <class 'torch.Tensor'>
tensor([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
    

##### MY EDIT OF THE ABOVE (EXPERIMENTATION)

In [13]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import rasterio
import numpy as np

# Define the custom dataset
class ImageDataset(Dataset):
    def __init__(self, image_dir, labels_df):
        self.image_dir = image_dir
        self.labels_df = labels_df

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df.index[idx]
        img_path = os.path.join(self.image_dir, f"{img_name}.tif")

        # Open the TIF image using rasterio
        with rasterio.open(img_path) as src:
            # Read the image data
            image_data = src.read()
        image_array = np.array(image_data)
        image_tensor = torch.from_numpy(image_array)
        image_tensor = image_tensor.float()

        label = self.labels_df.iloc[idx, 0]

        return image_tensor, label

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(32 * 50 * 50, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Set the paths and parameters
image_dir = "/Users/nadja/Documents/UU/Thesis/generated_tifs/"
labels_file = "/Users/nadja/Documents/UU/Thesis/generated_tifs/binary_palsa_labels.csv"
batch_size = 40
num_epochs = 5
learning_rate = 0.001

# Create the dataset and data loader
dataset = ImageDataset(image_dir, train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Create the CNN model
model = CNN()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "cnn_model.pth")


IndexError: Target 37 is out of bounds.